In [ ]:
#latency fix
BLUEZCARD=`pactl list cards short | egrep -o bluez.*[[:space:]]`
pactl set-card-profile $BLUEZCARD a2dp
pactl set-card-profile $BLUEZCARD hsp
pactl set-card-profile $BLUEZCARD a2dp

In [ ]:

#https://www.youtube.com/watch?v=zEhAJMQYSws
#https://wiki.archlinux.org/index.php/intel_graphics#Tearing
sudo pacman -S --noconfirm mesa lib32-mesa vulkan-intel vulkan-icd-loader ocl-icd lib32-ocl-icd intel-compute-runtime
sudo pacman -S --noconfirm xf86-video-intel

echo 'Section "Device"
  Identifier "Intel Graphics"
  Driver "modesetting"
  Option "AccelMethod" "glamor"
  Option "DRI" "3"
EndSection' | sudo tee "/etc/X11/xorg.conf.d/modesetting.conf"

echo 'Section "Device"
  Identifier "modesetting"
  Driver "modesetting"
EndSection' | sudo tee "/etc/X11/xorg.conf.d/modesetting.conf"

#ScreenTearing Fix
echo 'Section "Device"
  Identifier "Intel Graphics"
  Driver "intel"

  Option "TearFree" "true"
EndSection' | sudo tee /etc/X11/xorg.conf.d/20-intel.conf


In [ ]:
#intel/nvidia cuda
#https://askubuntu.com/questions/1061551/how-to-configure-igpu-for-xserver-and-nvidia-gpu-for-cuda-work
#https://gist.github.com/alexlee-gk/76a409f62a53883971a18a11af93241b
#https://www.youtube.com/watch?v=3BxAaaRDEEw
#Windows Gaming on Linux: Single GPU Passthrough Guide

lspci  | grep VGA
#00:02.0 VGA compatible controller: Intel Corporation Device 3e92
#01:00.0 VGA compatible controller: NVIDIA Corporation GP104 (rev a1)

#CUDA
#https://developer.nvidia.com/cuda-downloads?target_os=Linux&target_arch=x86_64&target_distro=Ubuntu&target_version=2004&target_type=debnetwork
wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64/cuda-ubuntu2004.pin
sudo mv cuda-ubuntu2004.pin /etc/apt/preferences.d/cuda-repository-pin-600
sudo apt-key adv --fetch-keys https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64/7fa2af80.pub
sudo add-apt-repository "deb http://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64/ /"
sudo apt-get update
sudo apt-get -y install cuda

In [ ]:
#integrated GPU/Cuda setup xorg
echo '# nvidia-settings: X configuration file generated by nvidia-settings
# nvidia-settings:  version 440.82

Section "ServerLayout"
    Identifier     "Layout0"
    Screen      0  "intel"
    Screen      1  "Screen0" 0 0
    InputDevice    "Keyboard0" "CoreKeyboard"
    InputDevice    "Mouse0" "CorePointer"
    Option         "Xinerama" "0"
EndSection

Section "Files"
EndSection

Section "Module"
    Load           "dbe"
    Load           "extmod"
    Load           "type1"
    Load           "freetype"
    Load           "glx"
EndSection

Section "InputDevice"
    # generated from default
    Identifier     "Mouse0"
    Driver         "mouse"
    Option         "Protocol" "auto"
    Option         "Device" "/dev/psaux"
    Option         "Emulate3Buttons" "no"
    Option         "ZAxisMapping" "4 5"
EndSection

Section "InputDevice"
    # generated from default
    Identifier     "Keyboard0"
    Driver         "kbd"
EndSection

Section "Monitor"
    # HorizSync source: edid, VertRefresh source: edid
    Identifier     "Monitor0"
    VendorName     "Unknown"
    ModelName      "AOC AG322FWG4"
    HorizSync       30.0 - 160.0
    VertRefresh     48.0 - 146.0
    Option         "DPMS"
EndSection


Section "Device"
    Identifier      "intel"
    Driver          "intel"
    BusId           "PCI:0@0:2:0"
    Option          "AccelMethod" "SNA"
EndSection

Section "Screen"
    Identifier      "intel"
    Device          "intel"
EndSection


Section "Device"
    Identifier     "Device0"
    Driver         "nvidia"
    VendorName     "NVIDIA Corporation"
    BoardName      "GeForce RTX 2080"
EndSection

Section "Screen"
    Identifier     "Screen0"
    Device         "Device0"
    Monitor        "Monitor0"
    DefaultDepth    24
    Option         "Stereo" "0"
    Option         "nvidiaXineramaInfoOrder" "DFP-1"
    Option         "metamodes" "nvidia-auto-select +0+0 {ForceCompositionPipeline=On, ForceFullCompositionPipeline=On}"
    Option         "SLI" "Off"
    Option         "MultiGPU" "Off"
    Option         "BaseMosaic" "off"
    SubSection     "Display"
        Depth       24
    EndSubSection
EndSection' | sudo tee /etc/X11/xorg.conf


In [ ]:
sudo pacman -S --noconfirm linux linux-firmware
echo "blacklist nouveau" | sudo tee -a /usr/lib/modprobe.d/nvidia.conf
echo "blacklist nouveau" | sudo tee -a /usr/lib/modprobe.d/blacklist.conf
echo "blacklist i2c_nvidia_gpu" | sudo tee -a /usr/lib/modprobe.d/blacklist.conf
sudo mkinitcpio -p linux56

echo '# nvidia-settings: X configuration file generated by nvidia-settings
# nvidia-settings:  version 440.82

Section "ServerLayout"
    Identifier     "Layout0"
    Screen      0  "intel"
    Screen      1  "Screen0" 0 0
EndSection

Section "Files"
EndSection

Section "Monitor"
    # HorizSync source: edid, VertRefresh source: edid
    Identifier     "Monitor0"
    VendorName     "Unknown"
    ModelName      "AOC AG322FWG4"
    HorizSync       30.0 - 160.0
    VertRefresh     48.0 - 146.0
    Option         "DPMS"
EndSection

Section "Device"
    Identifier      "intel"
    Driver          "intel"
    BusId           "PCI:0@0:2:0"
    Option          "AccelMethod" "SNA"
EndSection

Section "Screen"
    Identifier      "intel"
    Device          "intel"
EndSection


Section "Device"
    Identifier     "Device0"
    Driver         "nvidia"
    VendorName     "NVIDIA Corporation"
    BoardName      "GeForce RTX 2080"
EndSection

Section "Screen"
    Identifier     "Screen0"
    Device         "Device0"
    Monitor        "Monitor0"
    DefaultDepth    24
    Option         "Stereo" "0"
    Option         "nvidiaXineramaInfoOrder" "DFP-1"
    Option         "metamodes" "nvidia-auto-select +0+0 {ForceCompositionPipeline=On, ForceFullCompositionPipeline=On}"
    Option         "SLI" "Off"
    Option         "MultiGPU" "Off"
    Option         "BaseMosaic" "off"
    SubSection     "Display"
        Depth       24
    EndSubSection

EndSection' | sudo tee /etc/X11/xorg.conf


In [1]:

#NEW

#Hugepages
sudo apt install -y libhugetlbfs-bin
sudo kernelstub -a "intel_iommu=on"
sudo kernelstub -a "kvm.ignore_msrs=1"
sudo kernelstub -a "vfio-pci.ids=10de:1e82,10de:10f8"
echo "
vfio
vfio_iommu_type1
vfio_pci
vfio_virqfd
vhost-net
" | sudo tee -a /etc/initramfs-tools/modules
#modprobe vfio vfio_iommu_type1 vfio_pci vfio_virqfd vhost-net
cat /etc/initramfs-tools/modules
sudo update-initramfs -u

#https://www.reddit.com/r/VFIO/comments/ffrnf9/error_initially_creating_windows_10_vm/
#LibvirtHooks
sudo mkdir -p /etc/libvirt/hooks
sudo wget 'https://raw.githubusercontent.com/PassthroughPOST/VFIO-Tools/master/libvirt_hooks/qemu' \
     -O /etc/libvirt/hooks/qemu
sudo chmod u+x /etc/libvirt/hooks/qemu
sudo service libvirtd restart

#https://github.com/bryansteiner/gpu-passthrough-tutorial#part4
vmname="ChromeOS"
vmname="macOS"
vmname="win10-2"
# Before a VM is started, before resources are allocated:
sudo mkdir -p /etc/libvirt/hooks/qemu.d/$vmname/prepare/begin
# Before a VM is started, after resources are allocated:
sudo mkdir -p /etc/libvirt/hooks/qemu.d/$vmname/start/begin
# After a VM has started up:
sudo mkdir -p /etc/libvirt/hooks/qemu.d/$vmname/started/begin
# After a VM has shut down, before releasing its resources:
sudo mkdir -p /etc/libvirt/hooks/qemu.d/$vmname/stopped/end
# After a VM has shut down, after resources are released:
sudo mkdir -p /etc/libvirt/hooks/qemu.d/$vmname/release/end

#iommu.sh
#!/bin/bash
for d in /sys/kernel/iommu_groups/*/devices/*; do
  n=${d#*/iommu_groups/*}; n=${n%%/*}
  printf 'IOMMU Group %s ' "$n"
  lspci -nns "${d##*/}"
done

#kvm.conf
echo '## Virsh devices
VIRSH_GPU_VIDEO=pci_0000_01_00_0
VIRSH_GPU_AUDIO=pci_0000_01_00_1
VIRSH_GPU_USB=pci_0000_01_00_2
VIRSH_GPU_SERIAL=pci_0000_01_00_3
VIRSH_NVME_SSD=pci_0000_02_00_0' | sudo tee /etc/libvirt/hooks/kvm.conf
sudo chmod u+x /etc/libvirt/hooks/kvm.conf
#bind.sh
#unbind.sh
vmname="ChromeOS"
vmname="macOS"
vmname="win10-2"
sudo chmod +x /etc/libvirt/hooks/qemu
sudo chmod +x /etc/libvirt/hooks/kvm.conf
sudo chmod +x /etc/libvirt/hooks/qemu.d/win10-2/prepare/begin/bind_vfio.sh
sudo chmod +x /etc/libvirt/hooks/qemu.d/win10-2/release/end/unbind_vfio.sh

sudo modprobe vfio
sudo modprobe vfio_iommu_type1
sudo modprobe vfio_pci
sudo modprobe vfio_virqfd
sudo modprobe vhost-net
#sudo kernelstub -o "intel_iommu=on kvm.ignore_msrs=1 vfio-pci.ids=10de:1e82,10de:10f8" #PopOS
#https://mathiashueber.com/pci-passthrough-ubuntu-2004-virtual-machine/
#kvm.ignore_msrs=1 vfio-pci.ids=10de:1e82,10de:10f8
#sudo nano /etc/default/grub
#Systemd
echo "options kvm ignore_msrs=1" | sudo tee -a /etc/modprobe.d/kvm.conf
cat /etc/modprobe.d/kvm.conf
echo "options vfio_iommu_type1 allow_unsafe_interrupts=1" > sudo /etc/modprobe.d/iommu_unsafe_interrupts.conf
echo "blacklist radeon" >> sudo /etc/modprobe.d/blacklist.conf
echo "blacklist nouveau" >> sudo /etc/modprobe.d/blacklist.conf
echo "blacklist nvidia" >> sudo /etc/modprobe.d/blacklist.conf


lspci -v | grep VGA
lspci -n -s 01:00
01:00.0 0300: 10de:1e82 (rev a1)
01:00.1 0403: 10de:10f8 (rev a1)
01:00.2 0c03: 10de:10f8 (rev a1)
01:00.3 0c80: 10de:1ad9 (rev a1)
lspci -v | grep VGA
lspci -n -s 02:00
cat /sys/bus/pci/devices/0000:01:00.0/modalias
echo "options vfio-pci ids=10de:1e82,10de:10f8,10de:10f8,10de:1ad9,144d:a808 disable_vga=1"> sudo /etc/modprobe.d/vfio.conf
sudo update-initramfs -u -k all

#
#preferred
#https://heiko-sieger.info/running-windows-10-on-linux-using-kvm-with-vga-passthrough/
#Similar but shorter
#https://mathiashueber.com/windows-virtual-machine-gpu-passthrough-ubuntu/
sudo nano /etc/modprobe.d/local.conf
alias pci:v000010DEd00001E82sv0000103Csd0000372Bbc03sc00i00 vfio-pci
alias pci:v000010DEd000010F8sv0000103Csd0000372Bbc04sc03i00 vfio-pci
options vfio-pci ids=10de:1e82,10de:10f8



#CHECK
kvm-ok
lsmod | grep kvm
lsmod | grep vfio
qemu-system-x86_64 --version
lspci -kn | grep -A 2 01:00
dmesg | grep VFIO



SyntaxError: invalid syntax (<ipython-input-1-610e1f529947>, line 3)

In [ ]:
#https://github.com/intel/compute-runtime/releases
cd ~
mkdir neo
cd neo
wget https://github.com/intel/compute-runtime/releases/download/20.29.17408/intel-gmmlib_20.2.2_amd64.deb
wget https://github.com/intel/compute-runtime/releases/download/20.29.17408/intel-igc-core_1.0.4361_amd64.deb
wget https://github.com/intel/compute-runtime/releases/download/20.29.17408/intel-igc-opencl_1.0.4361_amd64.deb
wget https://github.com/intel/compute-runtime/releases/download/20.29.17408/intel-opencl_20.29.17408_amd64.deb
wget https://github.com/intel/compute-runtime/releases/download/20.29.17408/intel-ocloc_20.29.17408_amd64.deb
wget https://github.com/intel/compute-runtime/releases/download/20.29.17408/intel-level-zero-gpu_0.8.17408_amd64.deb
wget https://github.com/intel/compute-runtime/releases/download/20.29.17408/ww29.sum
sha256sum -c ww29.sum
sudo dpkg -i *.deb

In [ ]:
### RTL8812BU
# https://github.com/cilynx/rtl88x2bu
#sudo apt-get install -y dkms
#sudo pacman -S --noconfirm dkms
#sudo pacman -S --noconfirm linux56-headers
#sudo pacman-mirrors -f 5 && sudo pacman -Syyu
#sudo pacman -S $(pacman -Qsq "^linux" | grep "^linux[0-9]*[-rt]*$" | awk '{print $1"-headers"}' ORS=' ')
cd ~
git clone https://github.com/cilynx/rtl88x2bu.git
cd rtl88x2bu
VER=$(sed -n 's/\PACKAGE_VERSION="\(.*\)"/\1/p' dkms.conf)
sudo rsync -rvhP ./ /usr/src/rtl88x2bu-${VER}
sudo dkms add -m rtl88x2bu -v ${VER}
sudo dkms build -m rtl88x2bu -v ${VER}
sudo dkms install -m rtl88x2bu -v ${VER}
sudo modprobe 88x2bu

In [ ]:
### RTL8188EUS_ac
### easy setup but after one use unstable (fixed using mac address you dummy!)
# AIRCRACK-NG/RTL8188EUS_TLWN722N
# https://github.com/aircrack-ng/rtl8188eus
sudo apt update
sudo apt install bc -y
#https://unix.stackexchange.com/questions/574493/sudo-pacman-syu-fails-with-error-failed-to-commit-transaction-invalid-or-cor
sudo pacman -Sy archlinux-keyring
sudo pacman -Syu
sudo pacman -S binutils make gcc pkg-config fakeroot

cd ~
sudo rm -rf rtl8188eu
sudo rm -rf rtl8188eus
git clone https://github.com/aircrack-ng/rtl8188eus
cd rtl8188eus
#cp realtek_blacklist.conf /etc/modprobe.d/
#echo "blacklist r8188eu" > /etc/modprobe.d/realtek.conf
printf "\nblacklist r8188eu" | sudo tee -a /etc/modprobe.d/realtek.conf
make && sudo make install

#sudo mkdir -p /etc/modprobe.d/
#sudo touch /etc/modprobe.d/rtl8188eus.conf
#echo "options rtl8188eus rtw_power_mgnt=0 rtw_enusbss=0" | sudo tee /etc/modprobe.d/rtl8188eus.conf
printf "
# Configuration file for NetworkManager.
# See "man 5 NetworkManager.conf" for details.
[device]
wifi.scan-rand-mac-address=no

[ifupdown]
managed=false

[connection]
wifi.powersave=0

[main]
plugins=keyfile

[keyfile]
unmanaged-devices=d0:37:45:34:a2:64" | sudo tee -a /etc/NetworkManager/NetworkManager.conf
sudo rmmod 8188eu && sudo insmod 8188eu.ko


In [ ]:
###RTL8188EUS_qr
###larger range
#https://github.com/quickreflex/rtl8188eus
cd ~
sudo rm -rf rtl8188eu
sudo rm -rf rtl8188eus
git clone -b v5.2.2.4 https://github.com/quickreflex/rtl8188eus.git
cd rtl8188eus
make all
sudo make install

In [ ]:
### RTL8188EUS_lwf
###large range but somehow error after reboot
#https://github.com/lwfinger/rtl8188eu.git
cd ~
sudo rm -rf rtl8188eu
sudo rm -rf rtl8188eus
git clone https://github.com/lwfinger/rtl8188eu.git
cd rtl8188eu
make all
sudo make install
echo '[device]
wifi.scan-rand-mac-address=no' | sudo tee /etc/NetworkManager/conf.d/80-wifi.conf
sudo sed -i "s|blacklist r8188eu|#blacklist r8188eu|g" /etc/modprobe.d/realtek.conf

In [ ]:
### RTL8188FTV
#InstallRTL8188FTV
#https://askubuntu.com/questions/1062402/cant-find-wifi-drivers-for-0bdaf179-realtek-semiconductor-corp
cd ~
sudo apt-get install build-essential git dkms linux-headers-$(uname -r) -y
git clone https://github.com/kelebek333/rtl8188fu
sudo dkms add ./rtl8188fu
sudo dkms build rtl8188fu/1.0
sudo dkms install rtl8188fu/1.0
sudo cp ./rtl8188fu/firmware/rtl8188fufw.bin /lib/firmware/rtlwifi/

In [ ]:
#Brother MFC-L2703DW
#https://support.brother.com/g/b/downloadhowto.aspx?c=au&lang=en&prod=mfcl2703dw_as&os=128&dlid=dlf006893_000&flang=4&type3=625
cd ~
wget https://download.brother.com/welcome/dlf006893/linux-brprinter-installer-2.2.2-1.gz
sudo apt-get install -y p7zip
7z x linux-brprinter-installer-2.2.2-1.gz
sudo chmod u+x linux-brprinter-installer-*
sudo bash ./linux-brprinter-installer-*

In [ ]:
#Canon MG3660
#https://www.canon.com.au/support/sims-content?pid=e678bd22542d4167bff3c7f9aceb6178&cid=8A8FDAFBC57D4E3ABBAAA86BEF4CC487&ctype=dr
sudo apt-get install -y p7zip
cd ~
wget https://gdlp01.c-wss.com/gds/5/0100006905/01/scangearmp2-3.20-1-deb.tar.gz
tar xzvf scangearmp2*.gz
cd scangearmp2*deb
sudo ./install.sh
cd ~
wget https://gdlp01.c-wss.com/gds/2/0100006902/01/cnijfilter2-5.20-1-deb.tar.gz
tar xzvf cnijfilter2*.gz
cd cnijfilter2*deb
sudo ./install.sh


In [ ]:
#https://askubuntu.com/questions/984109/dolby-equivalent-for-ubuntu
#https://github.com/wwmm/pulseeffects/wiki/Package-Repositories#debian--ubuntu
#https://github.com/JackHack96/PulseEffects-Presets
sudo apt-get install -y pulseffects
#reboot
#pulseeffects
bash -c "$(curl -fsSL https://raw.githubusercontent.com/JackHack96/PulseEffects-Presets/master/install.sh)"
#convolver-wave-dolby atmos
#18.04
sudo add-apt-repository ppa:mikhailnov/pulseeffects -y
sudo apt update
sudo apt install pulseeffects pulseaudio --install-recommends


I've googled out a very good advise on forums that helps me to achieve a Dolby like sound on my Kubuntu 19.04 with Lenovo g780.

#Install PulseEffects https://github.com/wwmm/pulseeffects
#(repos with deb files are here: https://github.com/wwmm/pulseeffects/wiki/Package-Repositories#debian--ubuntu)
#Restart the user session or reboot after this, because PulseAudio will be upgraded, and it may cause problems if you don't restart.
Run PulseEffects and close it. It'll create all settings dirs on first launch. They required for next step.
Install PulseEffects-Presets from here: https://github.com/JackHack96/PulseEffects-Presets
(I've used the suggested script that automatically downloads them to PulseEffects import dirs, it will require flatpak that could be got from repos with sudo apt install flatpak)
Launch PulseEffects again. Select Convolver. Enable it. Click on wave button. You'll see a list of presets. Enable:
Dolby ATMOS ((128K MP3)) 1.Default.irs
Close the dialog and that's it. You can toggle Convolver in PulseEffects on and off while playng music to compare results. You may play with other presets as well.

For improving sound on a notebook or a tablet PulseEffects help pages come with a tuorial about how to achieve this.

App can be minimized to tray on GTK-enabled desktops with an additional application: https://github.com/boomshop/pulseffectstray

It's better enable autostart in app settings (it will copy it's desktop file to ~/.config/autostart with --gapplication-service command line. So next time start without GUI).

In [ ]:
#indicator
#https://www.omgubuntu.co.uk/2016/09/indicator-sound-switcher-makes-switching-audio-devices-ubuntu-snap#:~:text=To%20switch%20input%2Foutput%20on,sources%20with%20any%20major%20regularity.
sudo apt-add-repository -y ppa:yktooo/ppa
sudo apt update && sudo apt install -y indicator-sound-switcher
#sudo snap install audio-switcher

In [ ]:
echo "
default-sample-format = float32ne
default-sample-rate = 48000
alternate-sample-rate = 44100
default-sample-channels = 2
default-channel-map = front-left,front-right

default-fragments = 2
default-fragment-size-msec = 125
resample-method = speex-float-10
enable-lfe-remixing = no
high-priority = yes
nice-level = -11
realtime-scheduling = yes
realtime-priority = 9
rlimit-rtprio = 9
rlimit-rttime = -1
daemonize = no" | sudo tee /home/$USER/.config/pulse/daemon.conf
#soxr-vhq
#or if you want less latency speex-float-10 or speex-fixed-10

#https://medium.com/@gamunu/enable-high-quality-audio-on-linux-6f16f3fe7e1f
echo "default-sample-format = float32le
default-sample-rate = 48000
alternate-sample-rate = 44100
default-sample-channels = 2
default-channel-map = front-left,front-right
default-fragments = 2
default-fragment-size-msec = 125
resample-method = soxr-vhq
enable-lfe-remixing = no
high-priority = yes
nice-level = -11
realtime-scheduling = yes
realtime-priority = 9
rlimit-rtprio = 9
daemonize = no" | sudo tee /home/$USER/.config/pulse/daemon.conf


echo '
# Use PulseAudio plugin hw
pcm.!default {
type plug
slave.pcm hw
}' | sudo tee /etc/asound.conf

#Default
# Use PulseAudio by default
pcm.!default {
  type pulse
  fallback "sysdefault"
  hint {
    show on
    description "Default ALSA Output (currently PulseAudio Sound Server)"
  }
}

#Fix firefox no sound
#https://support.mozilla.org/en-US/questions/1252773
rm ~/.config/pulse/* 

In [ ]:

### Intel AC8265NGW
### Nvidia
#NVIDIA
#https://www.maketecheasier.com/install-nvidia-drivers-ubuntu/
sudo add-apt-repository ppa:graphics-drivers -y
sudo apt-get update
sudo apt-get install nvidia-driver-440 -y
sudo bash -c "echo blacklist nouveau > /etc/modprobe.d/blacklist-nvidia-nouveau.conf"
sudo bash -c "echo options nouveau modeset=0 >> /etc/modprobe.d/blacklist-nvidia-nouveau.conf"
cat /etc/modprobe.d/blacklist-nvidia-nouveau.conf
sudo update-initramfs -u


sudo su
sudo service lightdm stop ## if you are using 11.04 replace `lightdm` with `gdm`
chmod +x NVIDIA-Linux-x86_64-285.05.09.run
sudo sh NVIDIA-Linux-x86_64-285.05.09.run


#AIRCRACK-NG/RTL8812AU
sudo apt-get update
sudo apt-get install -y build-essential bc libelf-dev linux-headers-`uname -r`
cd ~
git clone -b v5.6.4.2 https://github.com/aircrack-ng/rtl8812au.git
cd rtl8812au
sudo ./dkms-install.sh
#sudo ./dkms-remove.sh

#For Raspberry (RPI)
#sudo apt-get install bc raspberrypi-kernel-headers

#Then run this step to change platform in Makefile, For RPI 1/2/3/ & 0/Zero:
#sed -i 's/CONFIG_PLATFORM_I386_PC = y/CONFIG_PLATFORM_I386_PC = n/g' Makefile
#sed -i 's/CONFIG_PLATFORM_ARM_RPI = n/CONFIG_PLATFORM_ARM_RPI = y/g' Makefile

#But for RPI 3B+ & 4B you will need to run those below which builds the ARM64 arch driver:
#sed -i 's/CONFIG_PLATFORM_I386_PC = y/CONFIG_PLATFORM_I386_PC = n/g' Makefile
#sed -i 's/CONFIG_PLATFORM_ARM64_RPI = n/CONFIG_PLATFORM_ARM64_RPI = y/g' Makefile

#In addition, if you receive an error message about unrecognized command #line option ‘-mgeneral-regs-only’ (i.e., Raspbian Buster), you will need to run the following commands:
#sed -i 's/^dkms build/ARCH=arm dkms build/' dkms-install.sh
#sed -i 's/^MAKE="/MAKE="ARCH=arm\ /' dkms.conf
